#<font color=yellow>15º Algoritmo Para 7 Espécies de Flores


##Este código ajustado aumenta a complexidade da rede neural adicionando mais camadas convolucionais, mais neurônios e técnicas de regularização (Batch Normalization e Dropout) para melhorar as métricas de desempenho. Além disso, a exibição gráfica foi ajustada para um tamanho maior para melhor visualização, e uma tabela foi adicionada para mostrar a quantidade de imagens novas classificadas e as métricas de desempenho para cada classe de flor.

##Além do citado no texto acima o caminho das imagens de flores para treino e teste da CNN e solicitado separadamente ao usuário para cada espécie de flor e em seguida após treinamento e teste e feito a solicitação do caminho para validação.

##NOTA - O algoritmo após o carregamento das imagens de flores para treino e teste da CNN não começa a realizar o treinamento devido ao fato de exeder o uso da quantidade de memória RAM da IDE do Google Colab, logo serão feitas auterações no algoritmo para resolver este problema. Pode ser que isto tenha ocorrido devido ao fato do posicionamento da rede neural, onde o aumento de neurônios distribuidos nas camadas internas aumentava de forma crescente da camada de entrada até a camada de saída.

In [ ]:
import numpy as np
from tensorflow import keras
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    balanced_accuracy_score,
    precision_score,
    f1_score,
    recall_score,
    roc_curve,
    auc,
    precision_recall_curve,
)
import os
from PIL import Image
import pandas as pd

# Montar o Google Drive
drive.mount('/content/drive')

# Defina as dimensões das imagens
img_height = 150
img_width = 150
num_classes = 7  # Sete classes para diferentes tipos de flores

# Dicionário de rótulos para cada tipo de flor
flower_labels = {
    "Margarida": 0,
    "Dente de Leão": 1,
    "Rosa": 2,
    "Girassol": 3,
    "Tulipa": 4,
    "Campânula": 5,
    "Lótus": 6,
}

def carregar_imagens_do_drive(caminho_pasta, flower_type):
    data = []
    labels = []
    if not os.path.exists(caminho_pasta):
        print(f"Erro: A pasta {caminho_pasta} não existe.")
        return data, labels
    for filename in os.listdir(caminho_pasta):
        img_path = os.path.join(caminho_pasta, filename)
        try:
            img = Image.open(img_path).resize((img_height, img_width)).convert('RGB')
            img_array = np.array(img)
            data.append(img_array)
            labels.append(flower_labels[flower_type])
        except Exception as e:
            print(f"Erro ao carregar imagem {img_path}: {e}")
    return data, labels

def carregar_todas_as_imagens():
    train_data = []
    train_labels = []
    val_data = []
    val_labels = []

    for flower_type in flower_labels.keys():
        caminho_treino = input(f"Digite o caminho da pasta de treino para {flower_type}: ")
        caminho_teste = input(f"Digite o caminho da pasta de teste para {flower_type}: ")

        data_treino, labels_treino = carregar_imagens_do_drive(caminho_treino, flower_type)
        data_teste, labels_teste = carregar_imagens_do_drive(caminho_teste, flower_type)

        train_data.extend(data_treino)
        train_labels.extend(labels_treino)
        val_data.extend(data_teste)
        val_labels.extend(labels_teste)

    return np.array(train_data), np.array(train_labels), np.array(val_data), np.array(val_labels)

# Carregar e processar as imagens para treinamento e validação
train_data, train_labels, val_data, val_labels = carregar_todas_as_imagens()

# Normalizar os dados de imagem
train_data = train_data / 255.0
val_data = val_data / 255.0

# ********************* IMPLEMENTAÇÃO DA REDE NEURAL ARTIFICIAL CNN LENET-5 **************************************************

# Construir o modelo LeNet-5 adaptado e aprimorado
model = keras.Sequential([
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu", input_shape=(img_height, img_width, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(num_classes, activation="softmax"),
])

# Compilar o modelo
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Treinar o modelo
history = model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=50, batch_size=32)

# ********************* FIM DO BLOCO DE CÓDIGO COM A REDE NEURAL CNN***************************************************

# Avaliar o modelo nos dados de teste
test_loss, test_accuracy = model.evaluate(val_data, val_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Prever as classes para os dados de teste
test_predictions = model.predict(val_data)
test_predictions_classes = np.argmax(test_predictions, axis=1)

# Salvar o modelo na pasta de downloads
model.save("/content/drive/My Drive/flower_classification_model.h5")

# Plotando gráfico de matriz de confusão para imagens de teste
confusion = confusion_matrix(val_labels, test_predictions_classes)
plt.figure(figsize=(10, 10))
sns.heatmap(
    confusion,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=flower_labels.keys(),
    yticklabels=flower_labels.keys(),
)
plt.xlabel("Predito")
plt.ylabel("Atual")
plt.title("Matriz de Confusão para Imagens de Teste")
plt.show()

# Calculando métricas de validação para imagens de teste
accuracy = accuracy_score(val_labels, test_predictions_classes)
balanced_accuracy = balanced_accuracy_score(val_labels, test_predictions_classes)
precision = precision_score(val_labels, test_predictions_classes, average='weighted')
f1 = f1_score(val_labels, test_predictions_classes, average='weighted')
recall = recall_score(val_labels, test_predictions_classes, average='weighted')

print("Acurácia para imagens de teste:", accuracy)
print("Acurácia Balanceada para imagens de teste:", balanced_accuracy)
print("Precisão para imagens de teste:", precision)
print("F1 Score para imagens de teste:", f1)
print("Recall para imagens de teste:", recall)

# Plotar gráficos de perda e acurácia
plt.figure(figsize=(20, 12))

# Gráfico de perda
plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Gráfico de acurácia
plt.subplot(2, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Transformar o problema em binário para uma abordagem "one-vs-rest"
for class_index in range(num_classes):
    print(f"\nClasse: {list(flower_labels.keys())[class_index]}")

    # Extrair rótulos binários para a classe atual
    binary_labels = (val_labels == class_index).astype(int)

    # Calcular probabilidades previstas para a classe atual
    predicted_probabilities = test_predictions[:, class_index]

    # Curva Precision-Recall
    precision, recall, thresholds = precision_recall_curve(binary_labels, predicted_probabilities)

    # F1-score para diferentes limiares de probabilidade
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)

    # Limiar que maximiza o F1-score
    optimal_threshold_index = np.argmax(f1_scores)

    # Limiar ótimo
    optimal_threshold = thresholds[optimal_threshold_index]
    optimal_precision = precision[optimal_threshold_index]
    optimal_recall = recall[optimal_threshold_index]

    print("Limiar ótimo que maximiza o F1-score (otimizando tanto a Precisão quanto o Recall):", optimal_threshold)
    print("Precisão ótima:", optimal_precision)
    print("Recall ótimo:", optimal_recall)

    # Plotando precision-recall curve
    plt.figure(figsize=(18, 10))

    plt.subplot(2, 3, 1)
    plt.plot(recall, precision, marker='.', label=f'Precision-Recall Curve for {list(flower_labels.keys())[class_index]}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve for {list(flower_labels.keys())[class_index]}')
    plt.legend()

    # Plotando precision VS threshold e recall VS threshold
    plt.subplot(2, 3, 2)
    plt.plot(thresholds, precision[:-1], label='Precision', color='blue')
    plt.plot(thresholds, recall[:-1], label='Recall', color='green')
    plt.xlabel('Threshold')
    plt.ylabel('Value')
    plt.title(f'Precision and Recall vs. Threshold for {list(flower_labels.keys())[class_index]}')
    plt.axvline(x=optimal_threshold, color='red', linestyle='--', label=f'Optimal Threshold ({optimal_threshold:.3f})')
    plt.legend()

    # Plotando F1-score VS threshold
    plt.subplot(2, 3, 3)
    plt.plot(thresholds, f1_scores[:-1], label='F1 Score', color='orange')
    plt.xlabel('Threshold')
    plt.ylabel('F1 Score')
    plt.title(f'F1 Score vs. Threshold for {list(flower_labels.keys())[class_index]}')
    plt.axvline(x=optimal_threshold, color='red', linestyle='--', label=f'Optimal Threshold ({optimal_threshold:.3f})')
    plt.legend()

    # Plotando curva ROC e AUC
    plt.subplot(2, 3, 4)
    fpr, tpr, _ = roc_curve(binary_labels, predicted_probabilities)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic (ROC) Curve for {list(flower_labels.keys())[class_index]}')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Função para identificar e exibir novas imagens
def identificar_e_exibir_imagens():
    caminho_novas_imagens = input("Digite o caminho da pasta de novas imagens no Google Drive: ")

    new_images = []
    filenames = []
    for filename in os.listdir(caminho_novas_imagens):
        img_path = os.path.join(caminho_novas_imagens, filename)
        img = Image.open(img_path).resize((img_height, img_width)).convert('RGB')
        img_array = np.array(img)
        new_images.append(img_array)
        filenames.append(filename)

    if not new_images:
        print("Nenhuma imagem carregada.")
        return

    new_images = np.array(new_images) / 255.0

    predictions = model.predict(new_images)
    predicted_classes = np.argmax(predictions, axis=1)
    confidences = np.max(predictions, axis=1) * 100

    num_cols = 5
    num_rows = (len(new_images) + num_cols - 1) // num_cols  # Calcular o número de linhas necessário

    plt.figure(figsize=(20, num_rows * 4))  # Ajustar o tamanho da figura

    for i, (filename, predicted_class, confidence) in enumerate(zip(filenames, predicted_classes, confidences)):
        plt.subplot(num_rows, num_cols, i + 1)
        img = Image.open(os.path.join(caminho_novas_imagens, filename))
        plt.imshow(img)
        plt.title(f'{list(flower_labels.keys())[predicted_class]} ({confidence:.2f}%)')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

    # Exibir quantidade de imagens novas classificadas e percentual de acurácia e precisão
    total_images = len(new_images)
    class_counts = np.bincount(predicted_classes, minlength=num_classes)
    class_accuracies = [np.sum(predicted_classes == i) / total_images for i in range(num_classes)]
    class_precisions = [precision_score(new_images, predicted_classes == i, average='weighted') for i in range(num_classes)]

    table_data = {
        "Classe de Flor": list(flower_labels.keys()),
        "Quantidade Classificada": class_counts,
        "Acurácia (%)": [acc * 100 for acc in class_accuracies],
        "Precisão (%)": [prec * 100 for prec in class_precisions],
    }
    df = pd.DataFrame(table_data)

    print("\nTabela de Classificação de Imagens Novas:")
    print(df.to_string(index=False))

# Função para o loop principal de identificação de imagens
def main_loop():
    while True:
        identificar_e_exibir_imagens()
        action = input("Digite 'novas' para identificar outras imagens ou 'sair' para terminar: ").lower()
        if action == 'sair':
            print("Encerrando o programa.")
            break

# Chamar a função para identificar novas imagens
main_loop()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Digite o caminho da pasta de treino para Margarida: /content/drive/MyDrive/Flores Treino/Margarida
Digite o caminho da pasta de teste para Margarida: /content/drive/MyDrive/Flores Teste/Margarida
Digite o caminho da pasta de treino para Dente de Leão: /content/drive/MyDrive/Flores Treino/Dente de Leão
Digite o caminho da pasta de teste para Dente de Leão: /content/drive/MyDrive/Flores Teste/Dente de Leão
Digite o caminho da pasta de treino para Rosa: /content/drive/MyDrive/Flores Treino/Rosa
Digite o caminho da pasta de teste para Rosa: /content/drive/MyDrive/Flores Teste/Rosa
Digite o caminho da pasta de treino para Girassol: /content/drive/MyDrive/Flores Treino/Girassol
Digite o caminho da pasta de teste para Girassol: /content/drive/MyDrive/Flores Teste/Girassol
Digite o caminho da pasta de treino para Tulipa: /content/drive/MyDrive/Flores Treino/Tulipa
